In [ ]:
import asyncio
from pydantic import BaseModel, Field
from typing import List, Literal
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnableMap, RunnableParallel

In [ ]:
# ── Step 1: Define Output Schema ─────────────────────────────────────────
class Issue(BaseModel):
    line: int = Field(..., description="line number (1-based)")
    severity: Literal["info", "warning", "error"]
    message: str
    suggestion: str

class ReviewResult(BaseModel):
    summary: str
    issues: List[Issue]

In [ ]:
# ── Step 2: Connect to Local LLM (llama-cpp) ─────────────────────────────
llm = ChatOpenAI(
    base_url="http://llama_cpp:8000/v1",
    api_key="local",
    model="Nous-Hermes-2-DPO_into_Nous_Hermes-2-Pro.Q8_0.gguf"
)

In [ ]:
# ── Step 3: Build Prompt & Output Parser ─────────────────────────────────
parser = PydanticOutputParser(pydantic_object=ReviewResult)

prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are a senior Python engineer. Act as a strict code reviewer.\n"
     "Instructions:\n"
     "1. Read the code the user sends.\n"
     "2. List any PEP 8, type-safety or design problems.\n"
     "3. Suggest concrete improvements.\n"
     "Return **only** JSON that matches the ReviewResult schema."),
    ("user", "{code}")
])

In [ ]:
# ── Step 4: Create Structured Output Chain ───────────────────────────────
chain = prompt | llm | parser

In [ ]:
# ── Step 5: Call Agent on Sample Code ────────────────────────────────────
async def run_code_review():
    code_snippet = """
def ADD(a: int,b: str):
  return a -  b
"""
    result: ReviewResult = await chain.ainvoke({"code": code_snippet})
    print(result.model_dump_json(indent=2))

In [ ]:
# ── Step 6: Run (Jupyter-compatible) ─────────────────────────────────────
await run_code_review()